In [1]:
import pandas as pd
import pickle 
import json
import TransformHelper

In [2]:
def save_to_disk(data, filepath):
    with open(filepath, 'wb') as file:
        pickle.dump(data, file)
def load_from_disk(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
        return data
def read_normalized_json_to_df(filepath):
    full_file_df = ""
    with open(filepath, 'r', encoding="utf-8") as json_file:
        json_full = json.load(json_file)
    full_file_df = pd.json_normalize(json_full)
    return full_file_df

### Building name list

In [3]:
# Read Json 
json_path = "./AllBuildingList_TaskAssignment.json"
with open(json_path, 'r', encoding="utf-8") as json_file:
    buildings_dict = json.load(json_file)

In [4]:
# Extract building names 
all_building_names = []
for elem in buildings_dict:
    all_building_names.append(elem["BuildingName"])
print("All Building names (" + str(len(all_building_names)) + ", unique names " + str(len(list(set((all_building_names))))) + "): \n", all_building_names)

All Building names (284, unique names 247): 
 ['Building_92', 'Garage_224', 'Building_226', 'Building_58', 'Building_182', 'Building_101', 'Building_225', 'Building_105', 'Building_139', 'Building_161', 'Garage_185', 'Building_146', 'Building_229', 'Building_86', 'Building_216', 'Building_215', 'Building_175', 'Building_196', 'Building_228', 'Building_124', 'Building_115', 'Building_192', 'Building_174', 'Building_219', 'Building_213', 'Building_192', 'Building_116', 'Building_116', 'Building_115', 'Building_189', 'Building_207', 'Building_193', 'Building_158', 'Building_123', 'Building_154', 'Building_125', 'Building_206', 'Building_191', 'Building_212', 'Building_190', 'Building_151', 'Building_84', 'Building_122', 'Building_106', 'Garage_185', 'Building_198', 'Building_131', 'Building_179', 'Building_201', 'Building_165', 'Building_184', 'Building_171', 'Building_176', 'Building_177', 'Building_208', 'Building_185', 'Building_178', 'Building_121', 'Building_157', 'Building_164', 'Bu

In [5]:
# Filter buildings
exclude_parts = [] # ["garage"]
filtered_names = []
for elem in all_building_names:
    for exclude in exclude_parts:
        if exclude in elem.lower():
            filtered_names.append(elem)
filtered_names = list(set(all_building_names) - set(filtered_names))
print("Filtered names (" + str(len(filtered_names)) + "): \n", filtered_names)

Filtered names (247): 
 ['Building_154', 'Building_176', 'Building_62', 'Building_227', 'Building_141', 'Building_179', 'Building_83', 'Building_60', 'Garage_86', 'TaskBuilding_17', 'Building_125', 'Building_182', 'Building_110', 'Church-TaskBuilding_16', 'Building_126', 'Garage_46', 'Building_105', 'Garage_72', 'TaskBuilding_55', 'Building_151', 'Building_103', 'Windmill-TaskBuilding_10_1', 'Building_69', 'Building_155', 'Building_91', 'Building_123', 'Building_220', 'Building_216', 'Building_228', 'Building_131', 'Building_234', 'Building_108', 'Building_222', 'Building_130', 'TaskBuilding_38', 'Building_132', 'Building_70', 'Building_199', 'Building_211', 'TaskBuilding_31', 'Building_157', 'Building_161', 'Building_189', 'Building_194', 'Building_112', 'Building_58', 'Building_204', 'Building_162', 'TaskBuilding_47', 'TaskBuilding_26', 'Building_233', 'Building_75', 'Building_187', 'Building_120', 'Building_215', 'Building_136', 'Building_89', 'Building_165', 'Garage_224', 'Building

In [6]:
# rename buildings
rename_dict = {'Garage_235':'Building_235', 'Garage_224':'Building_224', 'Garage_185':'Building_185', \
                'Garage_71':'Building_71', 'Garage_86':'Building_86', 'Garage_98':'Building_98', \
                'Garage_26':'TaskBuilding_26', 'Garage_46':'TaskBuilding_46', 'Garage_72':'Building_72'}
renamed_buildings = []
for elem in filtered_names:
    if elem in [*rename_dict]:
        renamed_buildings.append(rename_dict[elem])
    else:
        renamed_buildings.append(elem)
print("Renamed buildings (" + str(len(renamed_buildings)) + "): \n" + str(renamed_buildings))       



Renamed buildings (247): 
['Building_154', 'Building_176', 'Building_62', 'Building_227', 'Building_141', 'Building_179', 'Building_83', 'Building_60', 'Building_86', 'TaskBuilding_17', 'Building_125', 'Building_182', 'Building_110', 'Church-TaskBuilding_16', 'Building_126', 'TaskBuilding_46', 'Building_105', 'Building_72', 'TaskBuilding_55', 'Building_151', 'Building_103', 'Windmill-TaskBuilding_10_1', 'Building_69', 'Building_155', 'Building_91', 'Building_123', 'Building_220', 'Building_216', 'Building_228', 'Building_131', 'Building_234', 'Building_108', 'Building_222', 'Building_130', 'TaskBuilding_38', 'Building_132', 'Building_70', 'Building_199', 'Building_211', 'TaskBuilding_31', 'Building_157', 'Building_161', 'Building_189', 'Building_194', 'Building_112', 'Building_58', 'Building_204', 'Building_162', 'TaskBuilding_47', 'TaskBuilding_26', 'Building_233', 'Building_75', 'Building_187', 'Building_120', 'Building_215', 'Building_136', 'Building_89', 'Building_165', 'Building_2

In [8]:
# save data
#renamed_buildings.sort()
#save_to_disk(renamed_buildings,"./building_names.pickle")

filtered_names.sort()
save_to_disk(filtered_names,"./building_names.pickle")


### Building locations

In [10]:
# Read all buildings
building_df = read_normalized_json_to_df(json_path)

# Load transformation info 
transform_infos = load_from_disk("./map_image_transform_infos.pickle")
transform_matrix = transform_infos["perspective_transform_matrix"]
image_size = transform_infos["image_size"]

# Apply transformation 
building_df = TransformHelper.apply_perspective_transform_matrix(transform_matrix,img_size=image_size,dataframe=building_df,df_x_name="BuildingPosition.x",df_y_name="BuildingPosition.z",transform_x_name="transformed_buildingposition_x",transform_y_name="transformed_buildingposition_y")

# Save to disk (non-unique)
save_to_disk(building_df, "./buildings_nonunique_positions.pickle")


'''
# Group by BuildingName 
building_cnt = building_df.groupby("BuildingName").count()

print("Buildings of the same name with multiple different positions:")
print(building_cnt.loc[building_cnt["BuildingID"] != 1]["BuildingID"])
print()

# Extract unique positions (take first, and ignore others)
buildings_unique = building_df.drop_duplicates(subset="BuildingName",keep="first")
print("Number of unique building names: " + str(len(buildings_unique)))
'''




Calculating Perspective Transform...


100%|██████████| 284/284 [00:00<00:00, 5184.35it/s]


'\n# Group by BuildingName \nbuilding_cnt = building_df.groupby("BuildingName").count()\n\nprint("Buildings of the same name with multiple different positions:")\nprint(building_cnt.loc[building_cnt["BuildingID"] != 1]["BuildingID"])\nprint()\n\n# Extract unique positions (take first, and ignore others)\nbuildings_unique = building_df.drop_duplicates(subset="BuildingName",keep="first")\nprint("Number of unique building names: " + str(len(buildings_unique)))\n'

In [10]:
buildings_dict

[{'BuildingID': 92,
  'BuildingName': 'Building_92',
  'BuildingPosition': {'x': 19.67999267578125,
   'y': -0.9300003051757812,
   'z': 196.27000427246094},
  'BuildingRotation': {'x': 0.0, 'y': 262.75927734375, 'z': 0.0},
  'buildingRotationQuaternion': {'x': 0.0,
   'y': -0.7503461241722107,
   'z': 0.0,
   'w': 0.6610451936721802},
  'BuildingDirection': {'x': -0.9920253753662109,
   'y': 0.0,
   'z': -0.12603867053985596}},
 {'BuildingID': 224,
  'BuildingName': 'Garage_224',
  'BuildingPosition': {'x': -119.1593017578125,
   'y': -1.514312744140625,
   'z': -225.21221923828125},
  'BuildingRotation': {'x': 270.0, 'y': 210.45294189453125, 'z': 0.0},
  'buildingRotationQuaternion': {'x': 0.185711070895195,
   'y': 0.6822840571403503,
   'z': 0.6822839379310608,
   'w': -0.18571104109287262},
  'BuildingDirection': {'x': 0.0,
   'y': 1.0000001192092896,
   'z': -2.682209014892578e-07}},
 {'BuildingID': 226,
  'BuildingName': 'Building_226',
  'BuildingPosition': {'x': -3.81921720504